In [1]:
import time as clock
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from IPython.display import clear_output, display
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

In [2]:
DEVICE = 'cpu'

In [3]:
# configuration of simulator
# and parametrs of reservouir
niter = 100
perm = np.load('perm_2.npy')
nx0, nx1 = perm.shape
nx2 = 1
perm = np.reshape(perm, (nx0, nx1, nx2))
poro = 0.1 + np.zeros((nx0, nx1, nx2))

dx0 = 1.0 / nx0
dx1 = 1.0 / nx1
dx2 = 1.0 / nx2

pwat = 2.0
poil = 4.0
vr = 0.3
kwat = 1.0
koil = 0.3

pmin = 0.0
pmax = 1.0

t_final = 3.0
dt = t_final / niter

# Read from file

In [180]:
# sim_data: pres, swat, soil
sim_data = np.load("data-100-new-two-sigma/sim_100.npy").astype(np.float32)#[sim_nidexes]

pwat_list = np.load("data-100-new-two-sigma/pwat_100.npy").astype(np.float32)
poil_list = np.load("data-100-new-two-sigma/poil_100.npy").astype(np.float32)
kwat_list = np.load("data-100-new-two-sigma/kwat_100.npy").astype(np.float32)
koil_list = np.load("data-100-new-two-sigma/koil_100.npy").astype(np.float32)

In [181]:
niter = 100
nx, ny = 64, 64
t_final = 3.0
dt = t_final / niter
dx, dy = 1 / nx, 1 / ny

indexes_high = np.argwhere(sim_data[:, :, :, :, :, 1] > 0.1)
indexes_low = np.argwhere(sim_data[:, :, :, :, :, 1] > 0)

indexes_low = indexes_low[np.random.randint(0, len(indexes_low),len(indexes_high) // 2)]
indexes = np.vstack([indexes_low, indexes_high])

simulation_data = np.zeros((indexes.shape[0], 3))
for i, row in enumerate(indexes):
    simulation_data[i] = sim_data[row[0], row[1], row[2], row[3], row[4], :3]

x_list = indexes[:, 0] * dx
y_list = indexes[:, 1] * dy
t_list = indexes[:, 3] * dt

pwat_rand = pwat_list[indexes[:, -1]]
poil_rand = poil_list[indexes[:, -1]]
kwat_rand = kwat_list[indexes[:, -1]]
koil_rand = koil_list[indexes[:, -1]]

In [187]:
# rand_indexes_2000 = np.random.randint(0, simulation_data.shape[0], 2000)
# rand_indexes_1000 = np.random.randint(0, simulation_data.shape[0], 1000)
# rand_indexes_500 = np.random.randint(0, simulation_data.shape[0], 500)
# rand_indexes_250 = np.random.randint(0, simulation_data.shape[0], 250)

# np.save('train-indexes-2sigma-2000.npy', rand_indexes_2000)
# np.save('train-indexes-2sigma-1000.npy', rand_indexes_1000)
# np.save('train-indexes-2sigma-500.npy', rand_indexes_500)
# np.save('train-indexes-2sigma-250.npy', rand_indexes_250)
# rand_indexes
# np.save('train-indexes-2sigma-250.npy', rand_indexes)

rand_indexes_2000 = np.load('train-indexes-2sigma-2000.npy')
rand_indexes_1000 = np.load('train-indexes-2sigma-1000.npy')
rand_indexes_500 = np.load('train-indexes-2sigma-500.npy')
rand_indexes_250 = np.load('train-indexes-2sigma-250.npy')

In [188]:
rand_indexes = rand_indexes_250#np.random.randint(0, simulation_data.shape[0], 100)

simulation_data_train = torch.tensor(simulation_data.astype(np.float32))[rand_indexes].requires_grad_(True)

x = torch.tensor(x_list.astype(np.float32))[rand_indexes].requires_grad_(True)
y = torch.tensor(y_list.astype(np.float32))[rand_indexes].requires_grad_(True)
t = torch.tensor(t_list.astype(np.float32))[rand_indexes].requires_grad_(True)


pwat = torch.tensor(pwat_rand.astype(np.float32))[rand_indexes]
poil = torch.tensor(poil_rand.astype(np.float32))[rand_indexes]
kwat = torch.tensor(kwat_rand.astype(np.float32))[rand_indexes]
koil = torch.tensor(koil_rand.astype(np.float32))[rand_indexes]

points = torch.stack((t, x, y, pwat, poil, kwat, koil), -1).requires_grad_(True).to(DEVICE)

In [189]:
points.shape, simulation_data_train.shape

(torch.Size([250, 7]), torch.Size([250, 3]))

In [190]:
class ModifiedPINN(nn.Module):
    """
    Modified MLP architecture based on Wang et al. with U-Net style connections.
    Particularly effective for capturing sharp gradients in pressure/saturation.
    Key improvements:
    - Modified MLP with U and V paths (proven effective for PINNs)
    - Skip connections for gradient flow
    - Separate branches for different physics (pressure vs saturation)
    - Adaptive activation functions
    """
    
    def __init__(self, input_dim=7, hidden_dim=128, num_layers=6, output_dim=7):
        super(ModifiedPINN, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Modified MLP: Two parallel input branches (U and V paths)
        self.U = nn.Linear(input_dim, hidden_dim)
        self.V = nn.Linear(input_dim, hidden_dim)
        
        # Hidden layers with residual connections
        self.hidden_layers = nn.ModuleList()
        for i in range(num_layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
        # Learnable scaling parameters for outputs
        self.pressure_scale = nn.Parameter(torch.ones(1))
        self.saturation_scale = nn.Parameter(torch.ones(2))
        self.velocity_scale = nn.Parameter(torch.ones(4))
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Xavier initialization for better gradient flow."""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=1.0)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        """
        Forward pass with modified MLP architecture.
        
        Args:
            x: Input [batch, 7] -> (t, x, y, pwat, poil, kwat, koil)
        
        Returns:
            out: [batch, 7] -> (pres, soil, swat, uoil_x, uoil_y, uwat_x, uwat_y)
        """
        # Modified MLP: Two parallel branches
        U = torch.tanh(self.U(x))
        V = torch.tanh(self.V(x))
        
        # Element-wise multiplication (key feature of modified MLP)
        h = U * V
        
        # Pass through hidden layers with residual connections
        for i, layer in enumerate(self.hidden_layers):
            h_new = torch.tanh(layer(h))
            # Residual connection every 2 layers
            if i % 2 == 1 and i > 0:
                h = h + h_new  # Skip connection
            else:
                h = h_new
        
        # Output layer
        out = self.output_layer(h)
        
        # Split outputs
        pressure = out[:, 0:1]
        saturation_oil = out[:, 1:2]
        saturation_wat = out[:, 2:3]
        velocities = out[:, 3:7]
        
        # Apply constraints with learnable scales
        # Pressure: softplus for positivity
        pressure = F.softplus(pressure * self.pressure_scale, beta=1, threshold=20)
        
        # Saturations: sigmoid to enforce [0, 1] bounds
        saturation_oil = torch.sigmoid(saturation_oil * self.saturation_scale[0])
        saturation_wat = torch.sigmoid(saturation_wat * self.saturation_scale[1])
        
        # Enforce saturation constraint: S_o + S_w = 1
        # Normalize to ensure sum equals 1
        sat_sum = saturation_oil + saturation_wat + 1e-8
        saturation_oil = saturation_oil / sat_sum
        saturation_wat = saturation_wat / sat_sum
        
        # Velocities: can be positive or negative, use tanh scaled
        velocities = torch.tanh(velocities) * self.velocity_scale
        
        return torch.cat([pressure, saturation_oil, saturation_wat, velocities], dim=1)

model = ModifiedPINN(
   input_dim=7,
   hidden_dim=128,  # Try 128, 256
   num_layers=6,    # Try 5-8
   output_dim=7
).to(DEVICE)
model.load_state_dict(torch.load('pinn-test4-250-batched-new.pth',map_location='cpu'))

# bad pressure
# model = ModifiedPINN(
#    input_dim=7,
#    hidden_dim=64,
#    num_layers=4,
#    output_dim=7
# ).to(DEVICE)
# model.load_state_dict(torch.load('pinn-fullbatch-best-1k-sigmoid.pth',map_location='cpu'))

# bad pressure
# model = ModifiedPINN(
#    input_dim=7,
#    hidden_dim=128,
#    num_layers=4,
#    output_dim=7
# ).to(DEVICE)
# model.load_state_dict(torch.load('pinn-fullbatch-best-1k-sigmoid-2.pth',map_location='cpu'))

# bad pressure
# model = ModifiedPINN(
#    input_dim=7,
#    hidden_dim=128,
#    num_layers=5,
#    output_dim=7
# ).to(DEVICE)
# model.load_state_dict(torch.load('pinn-fullbatch-best-1k-sigmoid-4.pth',map_location='cpu'))

# bad pressure
# model = ModifiedPINN(
#    input_dim=7,
#    hidden_dim=64,
#    num_layers=5,
#    output_dim=7
# ).to(DEVICE)
# model.load_state_dict(torch.load('pinn-fullbatch-best-2k-sigmoid-2.pth',map_location='cpu'))

# bad pressure
# model = ModifiedPINN(
#    input_dim=7,
#    hidden_dim=128,
#    num_layers=4,
#    output_dim=7
# ).to(DEVICE)
# model.load_state_dict(torch.load('pinn-fullbatch-best-150-sigmoid-last.pth',map_location='cpu'))
# model.load_state_dict(torch.load('pinn-fullbatch-best-150-sigmoid.pth',map_location='cpu'))

<All keys matched successfully>

# SVM

In [191]:
def check_current_situation_others(svm_model, scaler, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = svm_model.predict(scaler.transform(test_points))

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()
    
def check_current_situation_xgb(xgb, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = xgb.predict(test_points)

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()

In [192]:
X_train = points.cpu().detach().numpy()
y_train = simulation_data_train.cpu().detach().numpy()

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

svr = SVR(kernel="rbf", C=1.0, epsilon=0.1, gamma="scale")
model_svr = MultiOutputRegressor(svr)
model_svr.fit(X_train, y_train)
# y_pred = svr.predict(X_test_scaled)
# print("MSE:", mean_squared_error(y_test, y_pred))
# print("R^2:", r2_score(y_test, y_pred))

MultiOutputRegressor(estimator=SVR())

In [193]:
# check_current_situation_others(svr, scaler, t=0.0)
# check_current_situation_others(svr, scaler, t=0.01)
# check_current_situation_others(svr, scaler, t=0.02)
# check_current_situation_others(svr, scaler, t=0.03)
# check_current_situation_others(svr, scaler, t=0.04)
# check_current_situation_others(svr, scaler, t=0.05)

# XGBoost

In [194]:
# Example data
xgb_regressor = xgb.XGBRegressor(tree_method='hist', verbosity=2)
xgb_regressor.fit(X_train, y_train)

[21:08:35] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (250, 7, 1750).


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [195]:
# check_current_situation_xgb(xgb_regressor, t=0)
# check_current_situation_xgb(xgb_regressor, t=0.01)
# check_current_situation_xgb(xgb_regressor, t=0.02)
# check_current_situation_xgb(xgb_regressor, t=0.03)
# check_current_situation_xgb(xgb_regressor, t=0.04)
# check_current_situation_xgb(xgb_regressor, t=0.05)

# Gaussian Processes Regression

In [196]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) \
         + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
gp.fit(X_train, y_train)
# y_pred, sigma = gp.predict(X_test, return_std=True)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10, normalize_y=True)

In [197]:
# check_current_situation_xgb(gp, t=0)
# check_current_situation_xgb(gp, t=0.01)
# check_current_situation_xgb(gp, t=0.02)
# check_current_situation_xgb(gp, t=0.03)
# check_current_situation_xgb(gp, t=0.04)
# check_current_situation_xgb(gp, t=0.05)

# Metrics Computation

In [198]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    median_absolute_error, 
    mean_absolute_percentage_error
)

def evaluate_multidim_models(models_dict, X_test, y_test, device='cpu'):
    """
    Evaluates models on multi-output regression (Target dim: [N, 3]).
    Output Columns: Pressure, Soil (Oil), Swat (Water).
    
    Parameters:
    - models_dict: Dictionary {'ModelName': model_object}
    - X_test: Test features
    - y_test: Test targets (Must be shape [N, 3])
    - device: 'cpu' or 'cuda' (for PyTorch model)
    
    Returns:
    - pd.DataFrame: Rows are (Variable_Metric), Columns are model names.
    """
    results = {}
    
    # 1. Define the specific output names
    target_names = ["Pressure", "Soil", "Swat"]
    
    # 2. Prepare Ground Truth (y_true) as numpy [N, 3]
    if torch.is_tensor(y_test):
        y_true = y_test.detach().cpu().numpy()
    elif isinstance(y_test, (pd.DataFrame, pd.Series)):
        y_true = y_test.to_numpy()
    else:
        y_true = np.array(y_test)
        
    # Basic shape validation
    if y_true.ndim != 2 or y_true.shape[1] != 3:
        raise ValueError(f"y_test must be shape [N, 3], got {y_true.shape}")

    # 3. Evaluate each model
    for name, model in models_dict.items():
        y_pred = None
        
        # --- Generate Predictions [N, 3] ---
        
        # PyTorch Logic
        if isinstance(model, nn.Module):
            model.eval()
            
            # Prepare input tensor
            if torch.is_tensor(X_test):
                X_tensor = X_test.to(device)
            elif isinstance(X_test, (pd.DataFrame, pd.Series)):
                X_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
            else:
                X_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
            
            with torch.no_grad():
                raw_pred = model(X_tensor)[:, [0, 2, 1]]
                y_pred = raw_pred.cpu().numpy() # Keep shape [N, 3]
                
        # Scikit-Learn Logic
        else:
            if torch.is_tensor(X_test):
                X_input = X_test.cpu().numpy()
            else:
                X_input = X_test
                
            y_pred = model.predict(X_input) # Expecting [N, 3] output

        # Ensure prediction shape matches truth
        if y_pred.shape != y_true.shape:
             raise ValueError(f"Model {name} output shape {y_pred.shape} mismatch with y_test {y_true.shape}")

        # --- Calculate Metrics per Column ---
        model_metrics = {}
        
        # Loop over the 3 output dimensions
        for i, label in enumerate(target_names):
            # Extract single column (1D arrays)
            y_t = y_true[:, i]
            y_p = y_pred[:, i]
            
            # Calculate Standard Metrics
            mse = mean_squared_error(y_t, y_p)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_t, y_p)
            r2 = r2_score(y_t, y_p)
            medae = median_absolute_error(y_t, y_p)
            mape = mean_absolute_percentage_error(y_t, y_p)
            
            # Store with prefixed keys (e.g., "Pressure_MSE")
            model_metrics[f"{label}_MSE"] = mse
            model_metrics[f"{label}_RMSE"] = rmse
            model_metrics[f"{label}_MAE"] = mae
            model_metrics[f"{label}_R2"] = r2
            model_metrics[f"{label}_MedAE"] = medae
            model_metrics[f"{label}_MAPE"] = mape

        results[name] = model_metrics

    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

In [199]:
model = ModifiedPINN(
   input_dim=7,
   hidden_dim=128,
   num_layers=6,
   output_dim=7
).to(DEVICE)
model.load_state_dict(torch.load('pinn-test4-250-batched-new.pth',map_location='cpu'))
# edu/ijcai26-hierarchical-loss-pinns/test4-2phase-disp-7d/pinn-fullbatch-best-1k-sigmoid-2.pth
# model = ModifiedPINN(
#    input_dim=7,
#    hidden_dim=128,  # Try 128, 256
#    num_layers=6,    # Try 5-8
#    output_dim=7
# ).to(DEVICE)
# model.load_state_dict(torch.load('pinn_fullbatch_500-pre-trained-4.pth',map_location='cpu'))
class RobustPINN(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=128, num_layers=6, output_dim=7):
        super(RobustPINN, self).__init__()
        
        # 1. Fourier Embedding Layer
        # We only embed space-time (t, x, y). Physical params (pwat, etc) usually skip this.
        # self.coord_embed = FourierEmbedding(input_dim=3, scale=5.0, mapping_size=16) # Adjust scale! Higher = sharper features
        
        # Input dim changes: (embedded coords) + (remaining params)
        self.feature_dim = 7#self.coord_embed.output_dim + (input_dim - 3)
        
        # 2. Fully Connected ResNet Backbone
        self.layers = nn.ModuleList()
        
        # First layer
        self.input_layer = nn.Linear(self.feature_dim, hidden_dim)
        
        # Hidden layers (ResNet style)
        for _ in range(num_layers):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
            
        # Output heads
        self.out_pressure = nn.Linear(hidden_dim, 1)
        self.out_saturation = nn.Linear(hidden_dim, 1) # Only predict Sw, So = 1 - Sw
        self.out_velocity = nn.Linear(hidden_dim, 4)
        
        self.activation = nn.Tanh() # Tanh is standard for PINNs. SiLU (Swish) is also good.

    def forward(self, x):
        # x assumed to be: [t, x, y, pwat, poil, kwat, koil]
        coords = x[:, 0:3]
        params = x[:, 3:]
        
        # Embed coordinates
        # embedded_coords = self.coord_embed(coords)
        
        # Concatenate back with physics parameters
        h = x#torch.cat([embedded_coords, params], dim=1)
        
        # Main Backbone
        h = self.activation(self.input_layer(h))
        
        for layer in self.layers:
            # ResNet block: y = x + Activation(Layer(x))
            h_new = self.activation(layer(h))
            h = h + h_new # Residual connection essential for deep networks
            
        # Outputs
        pressure = self.out_pressure(h)
        sw = self.out_saturation(h)
        velocities = self.out_velocity(h)
        
        # Constraints
        # Force Sw to be between 0 and 1 strictly
        sw = torch.sigmoid(sw) 
        so = 1.0 - sw
        
        # No sigmoid on pressure, let physics loss handle magnitude
        # Velocities: Standard output
        pressure = pressure / max(pressure)
        return torch.cat([pressure, so, sw, velocities], dim=1)
model = RobustPINN(
   input_dim=7,
   hidden_dim=64,
   num_layers=4,
   output_dim=7
).to(DEVICE)
# /Users/kirill/Downloads/pinn-fullbatch-best-100-sigmoid.pth
model.load_state_dict(torch.load('/Users/kirill/Downloads/pinn-fullbatch-best-100-sigmoid.pth',map_location='cpu'))
# model.load_state_dict(torch.load('pinn_fullbatch_500-pre-trained-4.pth',map_location='cpu'))


import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class KANLinear(nn.Module):
    """
    A single KAN Layer (Efficient implementation).
    Instead of w*x, it computes: w_base * SiLU(x) + w_spline * B-Spline(x)
    """
    def __init__(self, in_features, out_features, grid_size=5, spline_order=3, scale_noise=0.1, scale_base=1.0, scale_spline=1.0):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order
        
        # Grid parameters (h)
        h = (1.0 * 2.0) / grid_size # assuming input range [-1, 1]
        self.grid = torch.arange(-spline_order, grid_size + spline_order + 1) * h - 1.0
        self.register_buffer("grid_fixed", self.grid)

        # 1. Base Weight (The linear residual path)
        # Resembles a standard MLP connection but with SiLU activation built-in
        self.base_weight = nn.Parameter(torch.Tensor(out_features, in_features))
        
        # 2. Spline Weights (The learnable non-linear functions)
        self.spline_weight = nn.Parameter(torch.Tensor(out_features, in_features, grid_size + spline_order))
        
        # Scaling factors
        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        
        # Activation for the base path
        self.base_activation = nn.SiLU()
        
        self.reset_parameters()

    def reset_parameters(self):
        # Initialize base weights like generic linear layer
        nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        
        # Initialize spline weights with noise
        with torch.no_grad():
            noise = (torch.rand(self.grid_size + self.spline_order, self.in_features, self.out_features) - 1/2) * self.scale_noise / self.grid_size
            self.spline_weight.data.copy_(0.01 * noise.permute(2, 1, 0))

    def b_splines(self, x):
        """
        Compute B-spline bases for input x.
        x: [batch, in_features]
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        
        # Provide grid for the batch
        grid = self.grid_fixed  # [grid_size + 2*order + 1]
        x = x.unsqueeze(-1)     # [batch, in_features, 1]
        
        # Basis order 0
        bases = ((x >= grid[:-1]) & (x < grid[1:])).to(x.dtype)
        
        # Recurrence for higher orders
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:-(k + 1)]) / (grid[k:-1] - grid[:-(k + 1)]) * bases[:, :, :-1] +
                (grid[k + 1:] - x) / (grid[k + 1:] - grid[1:-k]) * bases[:, :, 1:]
            )
        
        assert bases.size() == (x.size(0), self.in_features, self.grid_size + self.spline_order)
        return bases.contiguous()

    def forward(self, x):
        # We expect x to be in range [-1, 1] approx. 
        # If x explodes, the splines won't work well.
        
        # 1. Base Path (Linear + SiLU)
        base_output = F.linear(self.base_activation(x), self.base_weight)
        
        # 2. Spline Path
        bases = self.b_splines(x) # [batch, in_dim, num_bases]
        
        # Compute: sum(weight * basis)
        spline_output = F.linear(bases.view(x.size(0), -1), self.spline_weight.view(self.out_features, -1))
        
        return base_output + spline_output

class KANPINN(nn.Module):
    """
    KAN Architecture specifically for Fluid PINN.
    Input: 7 (t, x, y, properties...)
    Output: 7 (Pressure, Saturation, Velocity...)
    """
    def __init__(self, input_dim=7, hidden_dim=16, output_dim=7, grid_size=10):
        super(KANPINN, self).__init__()
        
        # Normalize input coordinates to [-1, 1] internally to help Splines
        # Assuming typical domain: x,y in [0,64], t in [0,1]
        self.register_buffer('lb', torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
        self.register_buffer('ub', torch.tensor([1.0, 64.0, 64.0, 100.0, 100.0, 10.0, 10.0])) # Update with your max values!

        # KAN Layers
        # Note: Hidden dim can be small (8 or 16) compared to MLP (128)
        # grid_size=10 allows capturing reasonably sharp gradients.
        self.layer1 = KANLinear(input_dim, hidden_dim, grid_size=grid_size)
        self.layer2 = KANLinear(hidden_dim, hidden_dim, grid_size=grid_size)
        self.layer3 = KANLinear(hidden_dim, hidden_dim, grid_size=grid_size)
        self.layer_out = KANLinear(hidden_dim, output_dim, grid_size=grid_size)
        
        # Constraint parameters
        self.sat_steepness = nn.Parameter(torch.tensor(5.0)) # Helps fit shock waves
        self.pres_scale = nn.Parameter(torch.tensor(1.0))

    def forward(self, x):
        # 1. Normalize Inputs to [-1, 1] for KAN stability
        # x_norm = 2 * (x - lb) / (ub - lb) - 1
        # If you don't know exact bounds, using LayerNorm or Tanh on input helps
        x_norm = torch.tanh(x) # Simple normalization strategy for stability
        
        # 2. Forward Pass through KAN
        h = self.layer1(x_norm)
        h = self.layer2(h)
        h = self.layer3(h)
        out = self.layer_out(h)
        
        # 3. Apply Physical Constraints
        
        # Pressure: Positive and scaled
        # Softplus is better than ReLU for PDEs (smooth derivative)
        pres = F.softplus(out[:, 0:1]) * 10.0 * self.pres_scale
        
        # Saturation: Constrained [0, 1]
        # We predict Water Saturation (Sw)
        sw_raw = out[:, 2:3]
        sw = torch.sigmoid(sw_raw * self.sat_steepness)
        so = 1.0 - sw
        
        # Velocity: Unconstrained
        vel = out[:, 3:7]
        
        return torch.cat([pres, so, sw, vel], dim=1)
# model = KANPINN(input_dim=7, hidden_dim=16, output_dim=7, grid_size=10).to(DEVICE)

class DecoupledPINN(nn.Module):
    def __init__(self, input_dim=7):
        super().__init__()
        
        # Network 1: Handles Pressure (Smooth, Elliptic)
        # Needs Tanh for smoothness
        self.net_pressure = nn.Sequential(
            nn.Linear(input_dim, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 1) # Output Pressure only
        )
        
        # Network 2: Handles Saturation (Sharp, Hyperbolic)
        # Needs sharper activation or more layers, but keep it small for 100 points
        self.net_saturation = nn.Sequential(
            nn.Linear(input_dim, 50),
            nn.ELU(), # ELU or SELU often works better for shocks than Tanh
            nn.Linear(50, 50),
            nn.ELU(),
            nn.Linear(50, 50),
            nn.ELU(),
            nn.Linear(50, 1) # Output Saturation only
        )
        
        # Network 3: Velocity (Dependent on both)
        self.net_velocity = nn.Sequential(
            nn.Linear(input_dim, 30),
            nn.Tanh(),
            nn.Linear(30, 4)
        )

    def forward(self, x):
        # Independent predictions
        pres = torch.nn.functional.softplus(self.net_pressure(x))
        sw = torch.sigmoid(self.net_saturation(x))
        vel = self.net_velocity(x)
        
        so = 1.0 - sw
        
        return torch.cat([pres, so, sw, vel], dim=1)
model = DecoupledPINN().to(DEVICE)
# /Users/kirill/Downloads/pinn-fullbatch-best-100-doubled.pth
model.load_state_dict(torch.load('/Users/kirill/Downloads/pinn-fullbatch-best-250-doubled.pth',map_location='cpu'))
# model.load_state_dict(torch.load('/Users/kirill/Downloads/pinn-fullbatch-best-100-sigmoid-2.pth',map_location='cpu'))
# model.load_state_dict(torch.load('/Users/kirill/Downloads/pinn-fullbatch-best-100-sigmoid-2-2.pth',map_location='cpu'))

# model = KANPINN(input_dim=7, hidden_dim=16, output_dim=7, grid_size=10).to(DEVICE)
# model.load_state_dict(torch.load('/Users/kirill/Downloads/pinn-fullbatch-best-100-sigmoid-kan.pth',map_location='cpu'))

<All keys matched successfully>

In [200]:
indexes_high = np.argwhere(sim_data[:, :, :, :, :, 1] > 0.21)
indexes_low = np.argwhere(sim_data[:, :, :, :, :, 1] > 0.0)

indexes_low = indexes_low[np.random.randint(0, len(indexes_low),len(indexes_high) // 2)]
indexes = indexes_high#np.vstack([indexes_low, indexes_high])

simulation_data = np.zeros((indexes.shape[0], 3))
for i, row in enumerate(indexes):
    simulation_data[i] = sim_data[row[0], row[1], row[2], row[3], row[4], :3]

x_list = indexes[:, 0] * dx
y_list = indexes[:, 1] * dy
t_list = indexes[:, 3] * dt

pwat_rand = pwat_list[indexes[:, -1]]
poil_rand = poil_list[indexes[:, -1]]
kwat_rand = kwat_list[indexes[:, -1]]
koil_rand = koil_list[indexes[:, -1]]

rand_indexes = np.random.randint(0, simulation_data.shape[0], 1000)
simulation_data_train = torch.tensor(simulation_data.astype(np.float32)).requires_grad_(True)[rand_indexes]

x = torch.tensor(x_list.astype(np.float32)).requires_grad_(True)[rand_indexes]
y = torch.tensor(y_list.astype(np.float32)).requires_grad_(True)[rand_indexes]
t = torch.tensor(t_list.astype(np.float32)).requires_grad_(True)[rand_indexes]


pwat = torch.tensor(pwat_rand.astype(np.float32))[rand_indexes]
poil = torch.tensor(poil_rand.astype(np.float32))[rand_indexes]
kwat = torch.tensor(kwat_rand.astype(np.float32))[rand_indexes]
koil = torch.tensor(koil_rand.astype(np.float32))[rand_indexes]

points = torch.stack((t, x, y, pwat, poil, kwat, koil), -1).requires_grad_(True).to(DEVICE)

In [201]:
# from diffusion_equation import compute_solution
# pwat=1.5
# poil=2.0
# kwat=1.5
# koil = 0.3

# perm = np.load('perm_2.npy')
# nx0, nx1 = perm.shape
# nx2 = 1
# perm = np.reshape(perm, (nx0, nx1, nx2))
# poro = 0.1 + np.zeros((nx0, nx1, nx2))

# dx0 = 1.0 / nx0
# dx1 = 1.0 / nx1
# dx2 = 1.0 / nx2
# vr = 0.3

# if isinstance(pwat, float):
#     pwat_list = pwat * np.ones(64 * 64)
#     poil_list = poil * np.ones(64 * 64)
#     kwat_list = kwat * np.ones(64 * 64)
#     koil_list = koil * np.ones(64 * 64)

# pmin = 0.0
# pmax = 1.0

# dt = 0.21e-1
# niter = 100


# pres, swat, soil = compute_solution(perm, poro,
#                                     dx0, dx1, dx2, dt * niter, niter,
#                                     pwat, kwat, poil, koil, vr,
#                                     pmin=0.0, pmax=1.0)

# time_for_model = (niter * dt) * torch.ones(64 * 64)
# x_for_model = dx0 * torch.arange(64)
# y_for_model = dx1 * torch.arange(64)
# cartesian_points = torch.cartesian_prod(x_for_model, y_for_model)

# X_test = np.stack((
#     time_for_model, 
#     cartesian_points[:, 0], 
#     cartesian_points[:, 1],
#     pwat * np.ones(64 * 64),
#     poil * np.ones(64 * 64),
#     kwat * np.ones(64 * 64),
#     koil * np.ones(64 * 64)
# ), -1)  
# y_test = np.stack(
#     (
#        pres[:, :, :, -1].reshape(-1, 1)[:, 0], 
#         swat[:, :, :, -1].reshape(-1, 1)[:, 0], 
#         soil[:, :, :, -1].reshape(-1, 1)[:, 0]
#     ), -1
# )

In [202]:
X_test = points.cpu().detach().numpy()
y_test = simulation_data_train.cpu().detach().numpy()[:, [0, 1, 2]]

In [203]:
my_models = {
    'PINN': model,
    'XGB': xgb_regressor,
    'SVM': model_svr,
    'GP': gp
}

df_metrics = evaluate_multidim_models(my_models, X_test, y_test)
df_metrics

,PINN,XGB,SVM,GP
Pressure_MSE,0.000353,0.000069,0.003933,0.002838
Pressure_RMSE,0.018790,0.008288,0.062710,0.053277
Pressure_MAE,0.010594,0.004567,0.051135,0.041640
Pressure_R2,0.375926,0.878595,-5.951050,-4.017237
Pressure_MedAE,0.005803,0.002333,0.044506,0.033837
Pressure_MAPE,0.011171,0.004812,0.052680,0.043018
Soil_MSE,0.000796,0.001959,0.005988,0.004808
Soil_RMSE,0.028206,0.044264,0.077379,0.069337
Soil_MAE,0.017393,0.033697,0.060697,0.056299
Soil_R2,0.878391,0.700505,0.084753,0.265108
